NOTE: DEZE CODE IS NOG NIET VOLLEDIG. We willen twee hoeken ($\theta$ en $\varphi$) meten met behulp van een tweetal microfoons op dezelfde rij ($\theta$) een een tweetal op dezelfde kolom ($\varphi$), terwijl deze code je maar één hoek geeft op basis van één paar. De code voor beide hoeken is identiek, maar er moeten nog kleine aanpassingen worden gemaakt in het tweede deel van de code.

TOEGEVOEGDE NOTE: Vermoedelijk zit er een fout in dtime bepalen. Verder doet de code wat hij moet doen (behalve dat de waarde niet in het domein van arccos zit, maar dat komt door de fout in dtime). Check github onzecode/toevoeging.jpg voor extra informatie.

De code bestaat uit twee delen. Het eerste deel doet de meting en alle setup. Het tweede deel berekend de hoeken. Onderscheid tussen de delen is gemaakt zodat we niet een aparte meting hoeven te doen per hoek. We hebben op deze manier genoeg aan één meting.

DEEL 1: Run onderstaande code om een meting te doen. Pas de geluidssnelheid (v) en de afstanden tussen de microfoons (d) onderin dit code-blok aan zoals gewenst. 

In [ ]:
#>>>>FINDING AUDIO DEVICE
import sounddevice as sd

indev = sd.query_devices("UMA16v2")
print(f"Found input device {indev}")

#>>>>>SETTINGS
freq = 44000 #frequency
nchan = 16 #number of channels
sd.default.samplerate = freq
sd.default.channels = nchan
sd.default.device = 1 #this can be different. print sd.query_devices() to check.

#>>>>>RECORD
import numpy as np

npt = 220000 #total number of measurements per channel (TIME * FREQ)
dur = npt/freq

print(">>> Start recording for %5.2fs (%d samples per channel)" % (dur,npt))
rec = sd.rec(npt)
sd.wait()
untransposed = np.array(rec)
arec = np.transpose(untransposed) #<<-- THIS IS OUR DATA
print(">>> Done (%s x %s samples)" % (str(np.shape(arec)[0]), str(np.shape(arec)[1])))

In [ ]:
import matplotlib.pyplot as plt

#APPLYING FILTER
import scipy.signal as sps

def BandpassFilter(samples, freq, center_freq, width, order=5):
    nyquist = freq / 2
    low  = (center_freq - width/2) / nyquist
    high = (center_freq + width/2) / nyquist
    b, a = sps.butter(order, [low, high], btype='band')
    return sps.filtfilt(b, a, samples)

outsmp = np.zeros((npt, 16))
for i in range(0, 16):
    outsmp[i, :] = BandpassFilter(arec[i, :], freq, 1911, 475) #outsmp is the filtered data!

#CALCULATING THE TIME DIFFERENCE
def time_difference(mic1index, mic2index, arec, freq): #<<-- use 1-index
    #difference in datapoints

    maxmic1 = max(arec[mic1index - 1])
    dp1 = np.where(arec[mic1index - 1] == maxmic1)[0][0]
    maxmic2 = max(arec[mic2index - 1])
    dp2 = np.where([mic2index - 1] == maxmic2)
    dpd = dp1 - dp2 #<<-- NOTE1 THE SIGNIFICANE OF THE SIGN!

    #time difference
    dtime = dpd/freq
    return dtime

def makeplots(mic1index, mic2index, arec, outsmp):
    arecmaxmic1 = max(arec[mic1index - 1])
    adp1 = np.where(arec[mic1index - 1] == arecmaxmic1)[0][0]
    arecmaxmic2 = max(arec[mic2index - 1])
    adp2 = np.where(arec[mic2index - 1] == arecmaxmic2)[0][0]

    outsmpmaxmic1 = max(outsmp[mic1index - 1])
    odp1 = np.where(outsmp[mic1index - 1] == outsmpmaxmic1)[0][0]
    outsmpmaxmic2 = max(outsmp[mic2index - 1])
    odp2 = np.where(outsmp[mic2index - 1] == outsmpmaxmic2)[0][0]

    print("arec dpd: ", abs(adp1 - adp2))
    print("outsmp dpd: ", abs(odp1 - odp2))
    
    some_numbers = np.linspace(0, npt, npt)

    fig, axs = plt.subplots(2, 2)
    axs[0, 0].plot(some_numbers, arec[mic1index - 1, :])
    axs[0, 0].plot(adp1, arec[mic1index - 1, adp1], 'ro')
    axs[0, 0].set_title('Arec mic1')
    #axs[0, 0].set_xlim(adp1-50,adp1+50)
    #axs[0, 0].set_axis_off()
    axs[0, 1].plot(some_numbers, arec[mic2index - 1, :], 'tab:orange')
    axs[0, 1].plot(adp2, arec[mic2index - 1, adp2], 'bo')
    axs[0, 1].set_title('Arec mic2')
    #axs[0, 1].set_xlim(adp2-50,adp2+50)
    #axs[0, 1].set_axis_off()
    axs[1, 0].plot(some_numbers, outsmp[mic1index - 1, :], 'tab:green')
    axs[1, 0].plot(odp1, outsmp[mic1index - 1, odp1], 'go')
    axs[1, 0].set_title('Outsmp mic1')
    #axs[1, 0].set_xlim(odp1-50,odp1+50)
    #axs[1, 0].set_axis_off()
    axs[1, 1].plot(some_numbers, outsmp[mic2index - 1, :], 'tab:red')
    axs[1, 1].plot(odp2, outsmp[mic2index - 1, odp2], 'yo')
    axs[1, 1].set_title('Outsmp mic2')
    #axs[1, 1].set_xlim(odp2-50,odp2+50)
    #axs[1, 1].set_axis_off()

    for ax in axs.flat:
        ax.set(xlabel='x-label', ylabel='y-label')

    # Hide x labels and tick labels for top plots and y ticks for right plots.
    for ax in axs.flat:
        ax.label_outer()

#CALCULATING THE ANGLE
import math 

def angle(mic1index, mic2index, outsmp, freq):
    v = 342.55 #speed of sound (m/s)
    d = 0.042 #distance between mics (m)
    theta = math.acos((v * time_difference(mic1index, mic2index, outsmp, freq))/d)
    return theta


DEEL 2: Pas "EERSTE_MIC" en "TWEEDE_MIC" aan. Merk op dat de volgorde uitmaakt voor de interpretatie van de hoek. Lees hiervoor de comment onderin de code.

HORIZONTAAL: Kies voor "EERSTE_MIC_h" en "TWEEDE_MIC_h" twee microfoons die in dezelfde rij liggen.

In [ ]:
EERSTE_MIC_h = 5 #in 1-index
TWEEDE_MIC_h = 12 #in 1-index

makeplots(EERSTE_MIC_h, TWEEDE_MIC_h, arec, outsmp)

print("Deze hoek is ", angle(EERSTE_MIC_h, TWEEDE_MIC_h, outsmp, freq), " radialen.")

#>>>>>>NOTEEEEEE: An angle theta should be interpreted as follows:
#Draw a line between "EERSTE_MIC" and "TWEEDE_MIC". Rotate that line with *output* radians over "EERSTE_MIC". The line will point towards the source.


VERTICAAL: Kies voor "EERSTE_MIC_v" en "TWEEDE_MIC_v" twee microfoons die in dezelfde kolom liggen.

In [ ]:
EERSTE_MIC_v = 1 #in 1-index
TWEEDE_MIC_v = 2 #in 1-index

makeplots(EERSTE_MIC_v, TWEEDE_MIC_v, arec, outsmp)

print("Deze hoek is ", angle(EERSTE_MIC_v, TWEEDE_MIC_v, outsmp, freq), " radialen.")

#>>>>>>NOTEEEEEE: An angle theta should be interpreted as follows:
#Draw a line between "EERSTE_MIC" and "TWEEDE_MIC". Rotate that line with *output* radians over "EERSTE_MIC". The line will point towards the source.